## Recuperación ranqueada y vectorización de documentos (RRDV)

Importamos librerías de utilidad.

In [35]:
import os
from pathlib import Path
from typing import List
from collections import Counter
import xml.etree.ElementTree as ET

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import pandas as pd
import numpy as np

# Descargar recursos necesarios de NLTK
nltk.download('punkt')  # Tokenizador
nltk.download('stopwords')  # Stopwords

# Configuración de NLTK
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

Definimos nuestras funciones de preprocesamiento de tokens y texto. Usamos la librería de `nltk`.

In [36]:
# def token_preprocessing(tokens: List[str]) -> List[str]:
#     """
#     Realiza el preprocesamiento de texto, aplicando las siguientes transformaciones:
#     1. Convierte todos los tokens a minúsculas.
#     2. Elimina stopwords.
#     3. Elimina tokens que no son palabras (puntuación, números, etc.).
#     4. Aplica stemming con PorterStemmer.

#     Args:
#         tokens (List[str]): Lista de tokens a preprocesar.

#     Returns:
#         List[str]: Lista de tokens preprocesados.
#     """
#     # Pasa los tokens a minúsculas
#     tokens = [word.lower() for word in tokens]

#     # Elimina stopwords (asume texto en inglés)
#     stop_words = set(stopwords.words('english'))
#     tokens = [word for word in tokens if word not in stop_words]

#     # Filtrar tokens no alfabéticos (palabras que no empiezan con una letra)
#     tokens = [word for word in tokens if 'a' <= word[0] <= 'z']

#     # Aplica stemming
#     ps = PorterStemmer()
#     return [ps.stem(word) for word in tokens]

def token_preprocessing(tokens: List[str]) -> List[str]:
    import re
    """
    Realiza el preprocesamiento de texto, aplicando las siguientes transformaciones:
    1. Convierte todos los tokens a minúsculas.
    2. Elimina stopwords.
    3. Elimina tokens que no son palabras (puntuación, números, etc.).
    4. Aplica stemming con PorterStemmer.

    Args:
        tokens (List[str]): Lista de tokens a preprocesar.

    Returns:
        List[str]: Lista de tokens preprocesados.
    """
    # Pasa los tokens a minúsculas
    tokens = [word.lower() for word in tokens]

    # Elimina stopwords (asume texto en inglés)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    tokens = [re.sub(r'\W', '', word) for word in tokens]
    tokens = [re.sub(r'\s+[a-zA-Z]\s+', '', word) for word in tokens]
    tokens = [re.sub(r'\^[a-zA-Z]\s+', '', word) for word in tokens]
    tokens = [re.sub(r'[0-9]+', '', word) for word in tokens]

    tokens = [word for word in tokens if len(word) > 0]

    # Aplica stemming
    ps = PorterStemmer()
    return [ps.stem(word) for word in tokens]

def text_preprocessing(text: str) -> List[str]:
    """
    Preprocesa un texto mediante la tokenización y el preprocesamiento de tokens.

    Args:
        text (str): Texto a preprocesar.

    Returns:
        List[str]: Lista de tokens preprocesados.
    """
    # Tokenizar el texto usando el tokenizador de nltk
    tokens = word_tokenize(text)
    
    # Preprocesar los tokens
    return token_preprocessing(tokens)

Definimos una clase `Document`. Esta nos servirá para almacenar cada uno de los documentos en una estructura que nos permita acceder rápidamente al conteo de sus tokens.

In [37]:
class Document:
    def __init__(self, text: str, name: str = 'nameless'):
        """
        Crea un objeto del tipo Document.

        Args:
            text (str): El contenido del documento.
            name (str): El nombre del documento. Por defecto es 'nameless'.
        """
        self.text = text
        self.name = name

        # Preprocesar el texto y contar la frecuencia de cada token
        counter = Counter(text_preprocessing(text))
        
        # Almacenar los conteos de términos como una Serie de pandas
        self.term_counts = pd.Series(counter.values(), index=counter.keys())

    def __repr__(self):
        """
        Representación formal del objeto Document.
        """
        return str(self)

    def __str__(self):
        """
        Representación informal del objeto Document, devuelve el nombre del documento.
        """
        return self.name

La siguiente función se encarga de cargar los documentos, parsearlos y convertirlos en objetos de tipo `Document`.

In [38]:
def load_docs(docs_folder_path: Path) -> List[Document]:
    """
    Carga los documentos (con extensión .naf) en una carpeta especificada y crea objetos Document para cada uno.

    Args:
        docs_folder_path (Path): Ruta a la carpeta que contiene los archivos .naf.

    Returns:
        List[Document]: Una lista de objetos Document.
    """
    # Encuentra todos los archivos con extensión .naf en la carpeta especificada
    files = [f for f in os.listdir(docs_folder_path) if f.endswith('.naf')]

    # Lista para almacenar los documentos cargados
    docs = []

    # Procesar cada archivo .naf
    for file in files:
        # Parsea el archivo XML
        tree = ET.parse(os.path.join(docs_folder_path, file))
        root = tree.getroot()

        # Extrae el texto crudo del documento
        raw_text = root.find('.//raw').text

        # Extrae el nombre del archivo, omitiendo la extensión
        name = file.split('.')[1]

        # Crea un objeto Document y lo añade a la lista
        docs.append(Document(raw_text, name))

    return docs

In [39]:
all_docs = load_docs(Path("./data/docs-raw-texts"))

Definimos la clase para la **Recuperación Ranqueada y Vectorización de Documentos.** Como parte de sus atributos almacena el conteo de términos por documento, y su IDF y TF-IDF. Con estos se hará la búsqueda ranqueada, haciendo uso de la similitud coseno.

In [40]:
class RRDV:
    def __init__(self, docs: List[Document]):
        """
        Inicializa RRDV con una lista de documentos.

        Args:
            docs (List[Document]): Lista de objetos Document para construir el índice TF-IDF.
        """
        self.docs = docs
        
        # Crear un DataFrame con los conteos de términos para cada documento
        self.term_counts = pd.DataFrame({
            doc.name: doc.term_counts for doc in self.docs
        })
        self.term_counts.fillna(0, inplace=True)

        # Calcular la frecuencia de documentos para cada término
        self.document_count = (self.term_counts >= 1).sum(axis=1)

        # Calcular el IDF (Inverse Document Frequency)
        self.idf = np.log10(len(self.docs) / self.document_count)

        # Calcular TF-IDF (Term Frequency-Inverse Document Frequency)
        self.tfidf = np.log10(1 + self.term_counts).mul(self.idf, axis=0)

    @staticmethod
    def cosine_similarity(tfidf_doc_1: pd.Series, tfidf_doc_2: pd.Series | pd.DataFrame) -> pd.Series:
        """
        Calcula la similitud coseno entre dos vectores.

        Args:
            tfidf_doc_1 (pd.Series): Vector del primer documento.
            tfidf_doc_2 (pd.Series | pd.DataFrame): Vector TF-IDF del segundo documento o un DataFrame de vectores.

        Returns:
            pd.Series: Similitudes coseno entre tfidf_doc_1 y tfidf_doc_2.
        """
        return np.dot(tfidf_doc_1, tfidf_doc_2) / (
            np.linalg.norm(tfidf_doc_1) * np.linalg.norm(tfidf_doc_2, axis=0))

    def search(self, query_document: Document, min_similarity: float = 0.0) -> pd.DataFrame:
        """
        Realiza una búsqueda para encontrar documentos similares al documento de consulta.

        Args:
            query_document (Document): Documento de consulta para buscar similitudes.
            min_similarity (float): Umbral mínimo de similitud para filtrar resultados. Por defecto es 0.

        Returns:
            pd.DataFrame: DataFrame con documentos relevantes y sus similitudes.
        """
        # Filtrar términos en el vocabulario del índice
        in_vocab_term_counts = query_document.term_counts[query_document.term_counts.index.isin(self.idf.index)]
        
        # Calcular el TF-IDF para el documento de consulta
        query_tfidf = (np.log10(1 + in_vocab_term_counts) * self.idf).fillna(0)

        # Calcular la similitud coseno entre el documento de consulta y todos los documentos en el índice
        similarities = self.cosine_similarity(query_tfidf, self.tfidf)
        
        # Crear un DataFrame con los resultados de similitud
        results = pd.DataFrame({
            'similarity': similarities,
            'doc': self.docs
        }, index=self.tfidf.columns)
        
        # Ordenar los resultados por similitud de mayor a menor
        results.sort_values(by='similarity', ascending=False, inplace=True)

        # Filtrar los resultados por similitud mínima
        results = results[results['similarity'] > min_similarity]
        
        return results

    def evaluate_search(self, queries: List[Document], output_path: Path):
        """
        Evalúa las consultas y escribe los resultados en un archivo de salida.

        Args:
            queries (List[Document]): Lista de documentos de consulta para evaluar.
            output_path (Path): Ruta del archivo donde se guardarán los resultados.
        """
        with open(output_path, 'w') as output_file:
            for query in queries:
                relevant_docs = self.search(query_document=query)    
                result_texts = [f'{doc_name}:{row.similarity}' for doc_name, row in relevant_docs.iterrows()]
                output_file.write(f"{query.name}\t{','.join(result_texts)}\n")


In [41]:
rrdv = RRDV(all_docs)

In [42]:
all_queries = load_docs(Path("./data/queries-raw-texts"))
rrdv.evaluate_search(all_queries, output_path=Path("./data/RRDV-consultas_resultado"))